In [33]:
import pandas as pd
import numpy as np
import sys
import os
from os import listdir


pd.set_option('display.max_columns', 500)

os.chdir(os.getcwd().replace('experiments',''))
import config as CONFIG

In [34]:

import pandas as pd
import os

In [35]:
from src.data import *

In [36]:
os.getcwd()

'c:\\Users\\Robert\\Documents\\Projekte\\dev\\sport_betting'

In [129]:
results = pd.read_csv("./data/results/results.csv",index_col=0)
fixtures = pd.read_csv("./data/fixtures/update.csv",index_col=0)

df = results.merge(fixtures, left_on=['Home Team','Div'], right_on=['HomeTeam','Div'], how='inner')[['Div','Home Team', 'Away Team']+["Home Prob","Away Prob","Draw Prob"]+['AvgH','AvgD','AvgA']+["Fair Home Odd","Fair Draw Odd","Fair Away Odd"]]

df['AvgH/Fair Home Odd'] = df['AvgH'] / df['Fair Home Odd']
df['AvgD/Fair Draw Odd'] = df['AvgD'] / df['Fair Draw Odd']
df['AvgA/Fair Away Odd'] = df['AvgA'] / df['Fair Away Odd']

df_away = df[(df['AvgA/Fair Away Odd']>1)&(df["Away Prob"]>0.45)]
df_home = df[(df['AvgH/Fair Home Odd']>1.)&(df["Home Prob"]>0.45)]

# Create a copy of df_away and df_home to avoid the SettingWithCopyWarning
df_away = df_away.copy()
df_home = df_home.copy()

# Modify the copied dataframes
df_away['potential'] = df['AvgA/Fair Away Odd']
df_away["bet on"] = "away"

df_home['potential'] = df_home['AvgH/Fair Home Odd']
df_home["bet on"] = "home"

# Concatenate the modified dataframes
betting_slip = pd.concat([df_away, df_home], axis=0)[["Div", "Home Team", "Away Team", "bet on", "potential"]]
betting_slip.sort_values('potential',ascending=False)

,Div,Home Team,Away Team,bet on,potential
19,E0,Brighton,Newcastle,home,1.180556
67,EC,Dag and Red,Fylde,home,1.169014
172,T1,Buyuksehyr,Konyaspor,home,1.088372
133,P1,Vizela,Gil Vicente,home,1.082126
56,E3,Morecambe,Salford,away,1.081818
2,D1,Union Berlin,RB Leipzig,away,1.081340
90,F2,Laval,Caen,away,1.073171
139,SC0,Aberdeen,Hibernian,home,1.065728
97,G1,Panserraikos,Panetolikos,home,1.059908
42,E2,Blackpool,Wigan,home,1.058537


In [130]:
import pandas as pd

class BettingSlipGenerator:
    def __init__(self, results_file, fixtures_file):
        self.results_file = results_file
        self.fixtures_file = fixtures_file

    def read_data(self):
        # Read data from CSV files
        self.results = pd.read_csv(self.results_file, index_col=0)
        self.fixtures = pd.read_csv(self.fixtures_file, index_col=0)

    def merge_data(self):
        # Merge dataframes
        self.df = self.results.merge(self.fixtures, left_on=['Home Team', 'Div'], right_on=['HomeTeam', 'Div'], how='inner')[
            ['Div', 'Home Team', 'Away Team', 'Home Prob', 'Away Prob', 'Draw Prob', 'AvgH', 'AvgD', 'AvgA',
             'Fair Home Odd', 'Fair Draw Odd', 'Fair Away Odd']]

    def calculate_ratios(self):
        # Calculate additional columns
        self.df['AvgH/Fair Home Odd'] = self.df['AvgH'] / self.df['Fair Home Odd']
        self.df['AvgD/Fair Draw Odd'] = self.df['AvgD'] / self.df['Fair Draw Odd']
        self.df['AvgA/Fair Away Odd'] = self.df['AvgA'] / self.df['Fair Away Odd']

    def generate_betting_slip(self):
        # Filter data
        df_away = self.df[(self.df['AvgA/Fair Away Odd'] > 1) & (self.df["Away Prob"] > 0.45)]
        df_home = self.df[(self.df['AvgH/Fair Home Odd'] > 1) & (self.df["Home Prob"] > 0.45)]

        # Create copies of df_away and df_home to avoid the SettingWithCopyWarning
        df_away = df_away.copy()
        df_home = df_home.copy()

        # Modify the copied dataframes
        df_away['potential'] = df_away['AvgA/Fair Away Odd']
        df_away["bet on"] = "away"

        df_home['potential'] = df_home['AvgH/Fair Home Odd']
        df_home["bet on"] = "home"

        # Concatenate the modified dataframes
        betting_slip = pd.concat([df_away, df_home], axis=0)[["Div", "Home Team", "Away Team", "bet on", "potential"]]

        # Sort the betting slip by 'potential' in descending order
        betting_slip = betting_slip.sort_values('potential', ascending=False)

        return betting_slip

if __name__ == "__main__":
    # Create an instance of the BettingSlipGenerator class
    betting_slip_generator = BettingSlipGenerator("./data/results/results.csv", "./data/fixtures/update.csv")

    # Execute the steps to generate the betting slip
    betting_slip_generator.read_data()
    betting_slip_generator.merge_data()
    betting_slip_generator.calculate_ratios()
    generated_betting_slip = betting_slip_generator.generate_betting_slip()

    # Display the resulting betting slip
    print(generated_betting_slip)


     Div       Home Team       Away Team bet on  potential
19    E0        Brighton       Newcastle   home   1.180556
67    EC     Dag and Red           Fylde   home   1.169014
172   T1      Buyuksehyr       Konyaspor   home   1.088372
133   P1          Vizela     Gil Vicente   home   1.082126
56    E3       Morecambe         Salford   away   1.081818
2     D1    Union Berlin      RB Leipzig   away   1.081340
90    F2           Laval            Caen   away   1.073171
139  SC0        Aberdeen       Hibernian   home   1.065728
97    G1    Panserraikos     Panetolikos   home   1.059908
42    E2       Blackpool           Wigan   home   1.058537
15    D2  Greuther Furth        Hannover   home   1.058511
78    F1           Brest          Rennes   away   1.054455
132   P1        Casa Pia         Rio Ave   home   1.053398
9     D2   Holstein Kiel       Paderborn   home   1.050691
147  SC1     Inverness C     Dunfermline   home   1.041475
123   N1     Almere City          Zwolle   away   1.0384